In [1]:
import os,sys,glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import util
from utils import parsers
from tqdm import tqdm
from parsers import num2aa
from parsers import to1letter

### load rcsb pdb data

In [2]:
df = pd.read_csv('table.csv')

In [31]:
df.keys()

Index(['Id', 'Element', 'Ion', 'Binding family',
       'Amino acids or nucleotide residues names', 'All bound residues',
       'Is homomer', 'Number of bound amino acids or nucleotide residues',
       'Number of all bound residues', 'Number of coordinating chains',
       'Representative'],
      dtype='object')

In [71]:
df_homo = df[(df['Is homomer'] == 1) & ~(df['Amino acids or nucleotide residues names'].str.contains('DG')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DT')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DC')) & ~(df['Amino acids or nucleotide residues names'].str.contains('DA'))]#.tail(40)


In [73]:
df_homo['Amino acids or nucleotide residues names'].str.contains('DU').sum()
#'DG' in df['Amino acids or nucleotide residues names']

0

In [82]:
df_homo[df_homo['Number of coordinating chains']==4]

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative
39,1NC7-MG-1,MG,Mg²⁺,T4,THR.,THR.,True,4,4,4,True
40,1NC7-MG-2,MG,Mg²⁺,L4,LEU.,LEU.,True,4,4,4,True
41,1NC7-MG-3,MG,Mg²⁺,T4,THR.,HOH.,True,4,2,4,True
1041,4GGH-MG-1,MG,Mg²⁺,N4,ASN.,ASN.HOH.,True,4,6,4,True
1042,4GGH-MG-2,MG,Mg²⁺,N4,ASN.,ASN.HOH.,True,4,6,4,False
...,...,...,...,...,...,...,...,...,...,...,...
21717,7WM2-K-2,K,K⁺,T4,THR.,THR.,True,4,4,4,True
21718,7WM2-K-3,K,K⁺,G4,GLY.,GLY.,True,4,4,4,False
21764,8A35-RB-1,RB,Rb⁺,T4,THR.,THR.,True,4,4,4,False
21765,8A35-RB-2,RB,Rb⁺,T4,THR.,THR.,True,4,4,4,False


In [111]:
df_homo['pdb_id'] = df_homo['Id'].str.split('-', expand=True)[0]

/tmp/ipykernel_3732043/988947585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homo['pdb_id'] = df_homo['Id'].str.split('-', expand=True)[0]


In [115]:
df_homo.head(50)

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative,pdb_id
2,5LDB-MG-1,MG,Mg²⁺,T2,THR.,HOH.THR.,True,2,3,2,True,5LDB
3,2SHK-MG-1,MG,Mg²⁺,A1G2R2,ALA.ARG.GLY.,ALA.GLN.HOH.,True,5,5,2,False,2SHK
7,2R6T-MG-1,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
8,2R6T-MG-2,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
9,2R6T-MG-3,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.ILE.,True,2,3,2,False,2R6T
10,2R6X-MG-1,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
11,2R6X-MG-2,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
12,2R6X-MG-3,MG,Mg²⁺,D1I1,ASP.ILE.,ASP.ATP.HOH.ILE.,True,2,4,2,False,2R6X
13,2R8D-MG-1,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,True,2R8D
14,2R8D-MG-2,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,False,2R8D


In [119]:
#df_homo['pdb_id'].str.contains('4HHB').sum()

0

In [124]:
df_homo.to_csv('df_homomer.csv',index=False)

In [140]:
from urllib.request import urlretrieve
import requests
from tqdm import tqdm
import validators

In [123]:
url = 'https://files.rcsb.org/download/2AE8.pdb'
file = 'datasets/rcsb_pdbs/2AE8.pdb'
urlretrieve(url, file)

('datasets/rcsb_pdbs/2AE8.pdb', <http.client.HTTPMessage at 0x7f4aa1a3b9a0>)

In [159]:
logs = {'pdb_id':[], 'status':[]}
for pdb_id in tqdm(df_homo['pdb_id'].unique()):
    logs['pdb_id'].append(pdb_id)
    url = f'https://files.rcsb.org/download/{pdb_id}.pdb'
    file = f'datasets/rcsb_pdbs/{pdb_id}.pdb'
    r = requests.get(url)
    if '404' in str(r):
        logs['status'].append(0)
        continue
    urlretrieve(url,file)
    logs['status'].append(1)
logs = pd.DataFrame(logs)

  4%|▍         | 138/3199 [01:05<23:28,  2.17it/s] 

In [ ]:
logs.to_csv('datasets.logs')

In [155]:
# #pdb_id
# validators.url("https://files.rcsb.org/download/4QYK.pdb")
# url = "https://files.rcsb.org/download/4QYK.pdb"
# url = "https://files.rcsb.org/download/2IEC.pdb"
# requests.get(url)
# r = requests.get(url)


### analyze datasets

In [2]:
logs = pd.read_csv('datasets.logs')#, index_col=False)
df_homo = pd.read_csv('df_homomer.csv')
df = pd.read_csv('table.csv')
#pdb_ok = logs[logs['status']==1]['pdb_id']
#df_homo_filter = df_homo[df_homo['pdb_id'].isin(pdb_ok)]
df_homo_filter = pd.read_csv('df_homo_filter.csv')

In [7]:
# df_homo_filter['metal'] = df_homo_filter['Id'].str.split('-', expand=True)[1]
# df_homo_filter.to_csv('df_homo_filter.csv',index=False)

In [8]:
df_homo_filter['metal'].isna().sum()

0

In [45]:
# df_homo_filter.to_csv('df_homo_filter.csv', index=False)

In [5]:
# pdb_ok.str.contains('8FJD')

In [4]:
#drop row in df_homo if pdb_id is not in pdb_ok

In [9]:
df_homo_filter['pdb_id'].nunique()
#df_homo_filter

3132

In [3]:
# df_homo[df_homo['pdb_id'] =='7PGI']
# df[df['Id'].str.contains('7PGI')]['Amino acids or nucleotide residues names'].str.split('.')


In [5]:
util.get_ligand_num('datasets/rcsb_pdbs/2R36.pdb', 'NI', multi=True)#[0][1]#.split(',')

[['31', 'B', 'NI', array([-2.000e-03, -3.000e-03, -8.328e+00])],
 ['32', 'B', 'NI', array([  7.202,   6.801, -19.948])],
 ['31', 'D', 'NI', array([ 0.035, -0.117,  8.724])],
 ['32', 'D', 'NI', array([-9.533,  2.855, 20.337])]]

In [6]:
util.get_ligand_contacts('datasets/rcsb_pdbs/2R36.pdb',hetatm_num=31, hetatm='NI',hetatm_chain='B', cutoff=8.0, hotspot=['HIS'])
#'NI' in 'NI'

hetxyz [[-2.000e-03 -3.000e-03 -8.328e+00]]


[['10', 'B', 'HIS']]

In [11]:
# df_homo_filter['metal_binding_id'] = df_homo_filter['Id'].str.split('-', expand=True)[2]
# df_homo_filter.to_csv('df_homo_filter.csv', index=False)

In [13]:
# df_homo_filter

In [16]:
# [int(i) for i in df_homo_filter['Ion'].str.split()]#.[int(i) for i in test_string.split() if i.isdigit()]

In [40]:
id_debug = '4XLE'
debug = False
#for i,row in df_homo_filter.iterrows():
for pdb_id in df_homo_filter['pdb_id'].unique():
    if pdb_id != id_debug and debug:
        continue
    sample_df = df_homo_filter[df_homo_filter['pdb_id'] == pdb_id]
    row = sample_df[sample_df['Representative'] == 1].head(1)
    if len(row) == 0:
        row = sample_df.sort_values(by='metal_binding_id').head(1)
    hotspots_raw = row['Amino acids or nucleotide residues names'].values[0].split('.')
    hotspots_raw.remove('')
    hotspots = []
    for h in hotspots_raw:
        if h in util.THREE_TO_ONE.keys():
            hotspots.append(h)
    metal_name = row['metal'].values[0]
    metal_binding_id = row['metal_binding_id'].values[0]
    no_bound_residues = row['Number of bound amino acids or nucleotide residues'].values[0]
    motif_residue = []

    if hotspots==[]:
        continue #sample more cause oversampling, or non-canonical resi
    metal_infos = util.get_ligand_num(f'datasets/rcsb_pdbs/{pdb_id}.pdb', metal_name, multi=True)
    for metal_info in metal_infos:
        metal_num, metal_chain, metal_name, metal_xyz = metal_info
        motif = util.get_ligand_contacts(f'datasets/rcsb_pdbs/{pdb_id}.pdb', hetatm_num=metal_num, hetatm=metal_name,hetatm_chain=metal_chain, cutoff=6.0,hotspot=hotspots) # motif_resi_num, motif_chain, motif_resi_id
        motif_residue = [item[-1] for item in motif]
        if len(motif) >=  len(hotspots):# and :
            if set(hotspots) == set(motif_residue):
                break
            elif len(set(motif_residue)) == len(hotspots):
                break

    with open(f'datasets/rcsb_pdbs/{pdb_id}.pdb', 'r') as f:
        w_lines = []
        for line in f.readlines():
            if line.startswith('HETATM'):
                if line[17:21].strip() == metal_name and line[22:26].strip() == metal_num and line[21:22].strip() == metal_chain:
                    w_lines.append(line[:30] + '{:8.3f}'.format(0.0) + '{:8.3f}'.format(0.0) + '{:8.3f}'.format(0.0) + line[54:])
            if line.startswith('ATOM'):
                for m in motif:
                    #[line[22:26].strip(), str(line[21:22].strip()),line[17:21].strip()])
                    if line[22:26].strip()== m[0] and line[21:22].strip() == m[1] and line[17:21].strip()== m[2]:
                        line_xyz = np.array([float(line[30:38]), float(line[38:46]), float(line[46:54])])
                        w_lines.append(line[:30] + '{:8.3f}'.format(line_xyz[0]-metal_xyz[0]) + '{:8.3f}'.format(line_xyz[1]-metal_xyz[1]) + '{:8.3f}'.format(line_xyz[2]-metal_xyz[2]) + line[54:])

    with open(f'datasets/rotamer_motif/{pdb_id}_{metal_binding_id}.pdb', 'w') as f1:
        f1.writelines(w_lines)
    if pdb_id == id_debug and debug:
        break

In [37]:
row

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative,pdb_id,metal_binding_id,metal
3738,4XLE-NA-3,NaN,Na⁺,N1T1V1,ASN.THR.VAL.,ASN.HOH.THR.VAL.,True,3,6,2,True,4XLE,3,NA


In [72]:
# df_homo_filter[df_homo_filter['pdb_id'] == '1AEW'].sort_values

In [43]:
# bound_residues, pdb_id, metal_name
no_bound_residues, motif, m, motif_residue

(3, [], ['143', 'A', 'HIS'], [])

In [4]:
# metal_infos= [['401', 'A', 'MG', np.array([126.878, 123.014,  36.517])],
#   ['401', 'B', 'MG', np.array([136.216,  88.583,  29.722])],
#   ['401', 'C', 'MG', np.array([136.062,  95.724,  -5.958])],
#   ['401', 'D', 'MG', np.array([126.643, 130.136,   0.986])],
#   ['402', 'D', 'MG', np.array([121.658, 118.475,  -8.37 ])],
#   ['401', 'E', 'MG', np.array([ 30.586, 119.339,  -7.67 ])],
#   ['401', 'F', 'MG', np.array([ 26.262, 129.272,  26.918])],
#   ['401', 'G', 'MG', np.array([34.604, 95.578, 37.712])],
#   ['401', 'H', 'MG', np.array([38.928, 85.574,  3.026])]]
# pdb_id= '3N6Q'
# hotspots = ['GLU']

In [20]:
# pdb_id= '3H9F'
# metal_name = 'MG'
metal_infos = util.get_ligand_num(f'datasets/rcsb_pdbs/{pdb_id}.pdb', metal_name, multi=True)
metal_infos, pdb_id

([['1010', 'A', 'NA', array([49.953, 26.317, 55.249])],
  ['1011', 'A', 'NA', array([27.026, 50.465, 68.139])],
  ['1012', 'A', 'NA', array([26.147, 53.467, 24.209])]],
 '4XLE')

In [23]:
# pdb_id, metal_infos
hotspots, motif_residue

(['ASN', 'THR', 'VAL'], [])

In [24]:
motif_residue = []
for metal_info in metal_infos:
    metal_num, metal_chain, metal_name, metal_xyz = metal_info
    print(metal_num, metal_chain, metal_name, metal_xyz)
    motif = util.get_ligand_contacts(f'datasets/rcsb_pdbs/{pdb_id}.pdb', hetatm_num=metal_num, hetatm=metal_name,hetatm_chain=metal_chain, cutoff=8.0,hotspot=hotspots) # motif_resi_num, 
    motif_residue = [item[-1] for item in motif]
    print(motif)
    print(motif_residue)
    #     break


1010 A NA [49.953 26.317 55.249]
[['516', 'A', 'ASN'], ['517', 'A', 'ASN'], ['545', 'A', 'VAL']]
['ASN', 'ASN', 'VAL']
1011 A NA [27.026 50.465 68.139]
[['534', 'A', 'VAL'], ['567', 'A', 'VAL']]
['VAL', 'VAL']
1012 A NA [26.147 53.467 24.209]
[['249', 'A', 'ASN']]
['ASN']


In [14]:
# # metal_infos, 
# motif = util.get_ligand_contacts(f'datasets/rcsb_pdbs/{pdb_id}.pdb', hetatm_num=metal_num, hetatm=metal_name,hetatm_chain=metal_chain, cutoff=5.0,hotspot=hotspots)
# len(motif)
# motif

In [10]:
# only get last element of motif
motif
motif

[]

In [14]:
hetatm_num=metal_num
hetatm=metal_name
hetatm_chain=metal_chain
cutoff=5.0
ca_xyz = np.zeros(3)
cb_xyz = np.zeros(3)
hetatm_xyz = np.zeros(3)
# chain = chain.split(',')
# chain = [i.strip().upper() for i in chain]
with open(f'datasets/rcsb_pdbs/{pdb_id}.pdb', 'r') as f:
    for line in f.readlines():
        if line.startswith('ATOM'):
            # if line[21:22].strip() in chain:
            if line[13:15] =='CA':
                ca_xyz = np.vstack((ca_xyz,np.array([float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())])))
            elif line[13:15] =='CB':
                cb_xyz = np.vstack((cb_xyz,np.array([float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())])))
        elif line.startswith('HETATM'):
            
            if (line[17:21].strip() in hetatm) and ((line[22:26].strip()) == hetatm_num) and (line[21:22].strip() in hetatm_chain):
                
                hetatm_xyz = np.vstack((hetatm_xyz,np.array([float(line[30:38].strip()), float(line[38:46].strip()), float(line[46:54].strip())])))

In [15]:
hetatm_xyz

array([[ 0.   ,  0.   ,  0.   ],
       [38.928, 85.574,  3.026]])

In [6]:
pdb_id, metal_binding_id
motif = util.get_ligand_num(f'datasets/rcsb_pdbs/{pdb_id}.pdb', metal_name, multi=True)

In [13]:
df_homo_filter[df_homo_filter['pdb_id'] == '2R8D']

,Id,Element,Ion,Binding family,Amino acids or nucleotide residues names,All bound residues,Is homomer,Number of bound amino acids or nucleotide residues,Number of all bound residues,Number of coordinating chains,Representative,pdb_id,metal_binding_id
8,2R8D-MG-1,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,True,2R8D,1
9,2R8D-MG-2,MG,Mg²⁺,D3,ASP.,ASP.HOH.,True,3,6,2,False,2R8D,2


In [14]:
(df_homo_filter['Representative'] == 0).sum()

12503

In [37]:
#(df_homo_filter['Representative']==1).sum()
count = 0 
for pdb_id in (df_homo_filter['pdb_id'].unique()):
    if sum(df_homo_filter[(df_homo_filter['pdb_id'] == pdb_id)]['Representative']==1)==0:
        count+=1

### data extract


In [2]:
df_homo_filter = pd.read_csv('df_homo_filter.csv')
rotamer_motif = glob.glob('/home/lhtran/class/scaffold_squad/datasets/rotamer_motif/*.pdb')
len(rotamer_motif)

3117

In [3]:
df_homo_filter.keys()

Index(['Id', 'Element', 'Ion', 'Binding family',
       'Amino acids or nucleotide residues names', 'All bound residues',
       'Is homomer', 'Number of bound amino acids or nucleotide residues',
       'Number of all bound residues', 'Number of coordinating chains',
       'Representative', 'pdb_id', 'metal_binding_id', 'metal'],
      dtype='object')

In [44]:
periodic_table = pd.read_csv('data/periodic_table_1.csv')
df_homo_filter = pd.read_csv('data/df_homo_filter.csv')
rotamer_motif = glob.glob('/home/lhtran/class/scaffold_squad/datasets/rotamer_motif/*.pdb')
df_rotamer = {'pdb':[], 'pdb_id':[], 'metal':[], 'ion':[], 'bound_residue':[], 'num_bound_residue':[], 'paper_bound_residue':[], 'num_paper_bound_residue':[], 'num_bound_chain':[], 'motif_binding_xyz':[], 'num_NN_atom':[], 'NN_atom_properties':[]}
count = 0
## NN_atom_properties consists of (a dict)
## atom type - symbol in periodic table
## dist to metal - l2 distance
## electronegativity - pauling unit
## unshared electron - number of unshared electron of the atom once the octet is satisfied
## valency - chemistry
## size- van der waals radius
## ...
for pdb in tqdm(rotamer_motif):
    pdb_f = pdb.split('/')[-1]
    pdb_id = pdb_f.split('_')[0]
    metal = util.get_ligand(pdb)
    if metal == None:
        count+=1
        continue
    motif_binding_id = pdb_f.split('_')[1].split('.')[0]
    out = parsers.parse_pdb(pdb)
    #out = pd.DataFrame(out)
    chains = []
    for idx in out['pdb_idx']:
        chains.append(idx[0])
    chains = list(set(chains))
    bound_seq = out['seq']
    bound_residue = []
    for s in bound_seq:
        bound_residue.append(to1letter[num2aa[s]])
    NN_atoms = {'atom_type':[], 'l2_d':[], 'van_der_waals_radius':[], 'electronegativity':[], 'unshared_electron':[], 'valency':[]}
    for i, resi_xyz in enumerate(out['xyz']):
        for j, bb_xyz in enumerate(resi_xyz):
            if any(np.isnan(bb_xyz)):
                continue
            if np.linalg.norm(bb_xyz) <= 3.0: #consensus cutoff
                atom_type = out['atom_type'][i][j]
                atom_property = periodic_table[periodic_table['Symbol'] == atom_type]
                NN_atoms['atom_type'].append(atom_type)
                NN_atoms['l2_d'].append(np.linalg.norm(bb_xyz))
                NN_atoms['van_der_waals_radius'].append(atom_property['AtomicRadius'].values[0])
                NN_atoms['electronegativity'].append(atom_property['Electronegativity'].values[0])
                NN_atoms['valency'].append(atom_property['NumberOfValence'].values[0])
                NN_atoms['unshared_electron'].append(atom_property['NumberOfValence'].values[0] - (8 - atom_property['NumberOfValence'].values[0]))
    NN_atoms_aton_type_tmp = NN_atoms['atom_type']
    NN_atoms_aton_type_tmp = NN_atoms_aton_type_tmp.remove('C') if 'C' in NN_atoms_aton_type_tmp else NN_atoms_aton_type_tmp
    if NN_atoms_aton_type_tmp == None: # remove C-only interacting atoms, not representative
        count+=1
        continue
    id_df_homo = f'{pdb_id.upper()}-{metal.upper()}-{motif_binding_id}'
    row_df_homo_filter = df_homo_filter[df_homo_filter['Id'] == id_df_homo]
    df_rotamer['pdb'].append(pdb_f)
    df_rotamer['pdb_id'].append(pdb_id)
    df_rotamer['metal'].append(metal)
    df_rotamer['ion'].append(row_df_homo_filter['Ion'].values[0])
    df_rotamer['bound_residue'].append(bound_residue)
    df_rotamer['num_bound_residue'].append(len(bound_residue))
    df_rotamer['paper_bound_residue'].append(row_df_homo_filter['Amino acids or nucleotide residues names'].values[0])
    df_rotamer['num_paper_bound_residue'].append(row_df_homo_filter['Amino acids or nucleotide residues names'].values[0].count('.'))
    df_rotamer['num_bound_chain'].append(len(chains))
    df_rotamer['motif_binding_xyz'].append(out)
    df_rotamer['num_NN_atom'].append(len(NN_atoms['atom_type']))
    df_rotamer['NN_atom_properties'].append(NN_atoms)

df_rotamer = pd.DataFrame(df_rotamer)

100%|██████████| 3117/3117 [00:06<00:00, 508.24it/s]


In [46]:
df_rotamer#['motif_binding_xyz'][0]

,pdb,pdb_id,metal,ion,bound_residue,num_bound_residue,paper_bound_residue,num_paper_bound_residue,num_bound_chain,motif_binding_xyz,num_NN_atom,NN_atom_properties
0,1WWS_1.pdb,1WWS,CA,Ca²⁺,[Y],1,TYR.,1,1,"{'xyz': [[[-1.849 -6.217 5.614], [-1.722 -5.2...",1,"{'atom_type': ['O'], 'l2_d': [2.8696444], 'van..."
1,4K9N_1.pdb,4K9N,MG,Mg²⁺,"[N, L, R, N, L, R]",6,ARG.ASN.LEU.,3,2,"{'xyz': [[[ 5.206 -0.653 -3.413], [ 3.864 -1.0...",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2,2A0Q_1.pdb,2A0Q,K,K⁺,"[D, D, I, G]",4,ASP.GLY.ILE.,3,2,"{'xyz': [[[-4.524 -2.809 1.515], [-3.375 -2.5...",3,"{'atom_type': ['O', 'O', 'O'], 'l2_d': [2.9466..."
3,3E7Z_1.pdb,3E7Z,ZN,Zn²⁺,[H],1,HIS.,1,1,"{'xyz': [[[-4.842 3.076 -2.066], [-5.643 1.8...",1,"{'atom_type': ['N'], 'l2_d': [2.068424], 'van_..."
4,7MCI_1.pdb,7MCI,CA,Ca²⁺,"[D, D, R, E]",4,ARG.ASP.GLU.,3,2,"{'xyz': [[[-1.000e-03 5.978e+00 -3.136e+00], ...",4,"{'atom_type': ['O', 'O', 'O', 'O'], 'l2_d': [2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2234,1C3H_1.pdb,1C3H,CA,Ca²⁺,"[N, V, D, Q, V, D, Q, Q, N, D]",10,ASN.ASP.GLN.VAL.,4,2,"{'xyz': [[[3.755 0.878 4.795], [2.792 0.666 3....",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2235,7RWX_1.pdb,7RWX,ZN,Zn²⁺,"[H, H, H]",3,HIS.,1,2,"{'xyz': [[[-6.217 -2.745 -2.579], [-5.069 -1.9...",2,"{'atom_type': ['N', 'N'], 'l2_d': [2.1764033, ..."
2236,6PN8_1.pdb,6PN8,ZN,Zn²⁺,"[C, C, C, C]",4,CYS.,1,2,"{'xyz': [[[-3.448 2.842 -3.131], [-3.314 3.0...",4,"{'atom_type': ['S', 'S', 'S', 'S'], 'l2_d': [2..."
2237,6SAG_1.pdb,6SAG,CA,Ca²⁺,"[N, P]",2,ASN.GLU.PRO.,3,1,"{'xyz': [[[3.386 0.054 5.462], [2.511 0.171 4....",2,"{'atom_type': ['O', 'O'], 'l2_d': [2.544683, 2..."


In [48]:
# df_rotamer = pd.read_csv('df_rotamer.csv')

In [49]:
df_rotamer.to_pickle('data/df_rotamer.pkl')
df_rotamer.to_csv('data/df_rotamer.csv')

In [2]:
df_rotamer_pkl = pd.read_pickle('data/df_rotamer.pkl')

In [39]:
type(df_rotamer_pkl['motif_binding_xyz'][0])
#tmp = df_rotamer_pkl['motif_binding_xyz'].map(ast.literal_eval)

dict

In [3]:
df_rotamer_pkl['motif_binding_xyz'][0]

{'xyz': array([[[-1.849, -6.217,  5.614],
         [-1.722, -5.22 ,  4.553],
         [-0.399, -4.468,  4.672],
         [ 0.209, -4.082,  3.667],
         [-2.889, -4.227,  4.63 ],
         [-2.819, -3.114,  3.601],
         [-2.095, -1.942,  3.848],
         [-3.494, -3.226,  2.386],
         [-2.053, -0.902,  2.903],
         [-3.459, -2.201,  1.439],
         [-2.744, -1.043,  1.705],
         [-2.759, -0.016,  0.789],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan],
         [   nan,    nan,    nan]]], dtype=float32),
 'mask': array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
          True,  True,  True, False, False, False, Fals

In [103]:
# #row_df_homo_filter['Amino acids or nucleotide residues names'].values[0].count('.')#[0])
# df_rotamer['ion']#['NN_atom_type']#[df_rotamer['pdb'] == '2WSB_1.pdb']['NN_atom_properties'].values[0]

In [4]:
pdbs = glob.glob('/home/lhtran/class/scaffold_squad/datasets/rcsb_pdbs/*.pdb')

In [5]:
df_rotamer_pkl

,pdb,pdb_id,metal,ion,bound_residue,num_bound_residue,paper_bound_residue,num_paper_bound_residue,num_bound_chain,motif_binding_xyz,num_NN_atom,NN_atom_properties
0,1WWS_1.pdb,1WWS,CA,Ca²⁺,[Y],1,TYR.,1,1,"{'xyz': [[[-1.849 -6.217 5.614], [-1.722 -5.2...",1,"{'atom_type': ['O'], 'l2_d': [2.8696444], 'van..."
1,4K9N_1.pdb,4K9N,MG,Mg²⁺,"[N, L, R, N, L, R]",6,ARG.ASN.LEU.,3,2,"{'xyz': [[[ 5.206 -0.653 -3.413], [ 3.864 -1.0...",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2,2A0Q_1.pdb,2A0Q,K,K⁺,"[D, D, I, G]",4,ASP.GLY.ILE.,3,2,"{'xyz': [[[-4.524 -2.809 1.515], [-3.375 -2.5...",3,"{'atom_type': ['O', 'O', 'O'], 'l2_d': [2.9466..."
3,3E7Z_1.pdb,3E7Z,ZN,Zn²⁺,[H],1,HIS.,1,1,"{'xyz': [[[-4.842 3.076 -2.066], [-5.643 1.8...",1,"{'atom_type': ['N'], 'l2_d': [2.068424], 'van_..."
4,7MCI_1.pdb,7MCI,CA,Ca²⁺,"[D, D, R, E]",4,ARG.ASP.GLU.,3,2,"{'xyz': [[[-1.000e-03 5.978e+00 -3.136e+00], ...",4,"{'atom_type': ['O', 'O', 'O', 'O'], 'l2_d': [2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2234,1C3H_1.pdb,1C3H,CA,Ca²⁺,"[N, V, D, Q, V, D, Q, Q, N, D]",10,ASN.ASP.GLN.VAL.,4,2,"{'xyz': [[[3.755 0.878 4.795], [2.792 0.666 3....",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2235,7RWX_1.pdb,7RWX,ZN,Zn²⁺,"[H, H, H]",3,HIS.,1,2,"{'xyz': [[[-6.217 -2.745 -2.579], [-5.069 -1.9...",2,"{'atom_type': ['N', 'N'], 'l2_d': [2.1764033, ..."
2236,6PN8_1.pdb,6PN8,ZN,Zn²⁺,"[C, C, C, C]",4,CYS.,1,2,"{'xyz': [[[-3.448 2.842 -3.131], [-3.314 3.0...",4,"{'atom_type': ['S', 'S', 'S', 'S'], 'l2_d': [2..."
2237,6SAG_1.pdb,6SAG,CA,Ca²⁺,"[N, P]",2,ASN.GLU.PRO.,3,1,"{'xyz': [[[3.386 0.054 5.462], [2.511 0.171 4....",2,"{'atom_type': ['O', 'O'], 'l2_d': [2.544683, 2..."


In [6]:
out = parsers.parse_pdb(pdbs[1])
# for i, resi_xyz in enumerate(out['xyz']):
#     for j, bb_xyz in enumerate(resi_xyz):
#         if any(np.isnan(bb_xyz)):
#             continue
#         if np.linalg.norm(bb_xyz) <= 3.0: #consensus cutoff
#             atom_type = out['atom_type'][i][j]
#             print(atom_type, j)

In [16]:
pdbs[1]

'/home/lhtran/class/scaffold_squad/datasets/rcsb_pdbs/4IMS.pdb'

In [15]:
for i in out['info_het']:
    if i['name'] == 'ZN':
        print(i)

{'idx': 6758, 'atom_id': 'ZN', 'atom_type': 'ZN', 'name': 'ZN'}
{'idx': 6759, 'atom_id': 'ZN', 'atom_type': 'ZN', 'name': 'ZN'}
{'idx': 6839, 'atom_id': 'ZN', 'atom_type': 'ZN', 'name': 'ZN'}


In [105]:
# NN_atoms['atom_type']#.remove('C') == 

In [106]:
# out['xyz'][0], out['atom_type'][0]

In [107]:
# out['xyz_het']

In [35]:
# #out['atom_type']#[2], i
out = parsers.parse_pdb('/home/lhtran/class/scaffold_squad/datasets/rotamer_motif/4DLX_1.pdb')
out = parsers.parse_pdb('/home/lhtran/class/scaffold_squad/datasets/rcsb_pdbs/7R2U.pdb')
# #f'{pdb_id.upper()}-{metal.upper()}-{motif_binding_id}'
# metal == None

In [39]:
df_rotamer_pkl[['pdb_id','metal','ion','num_bound_residue','num_paper_bound_residue','num_bound_chain','num_NN_atom']]


,pdb_id,metal,ion,num_bound_residue,num_paper_bound_residue,num_bound_chain,num_NN_atom
0,1WWS,CA,Ca²⁺,1,1,1,1
1,4K9N,MG,Mg²⁺,6,3,2,6
2,2A0Q,K,K⁺,4,3,2,3
3,3E7Z,ZN,Zn²⁺,1,1,1,1
4,7MCI,CA,Ca²⁺,4,3,2,4
...,...,...,...,...,...,...,...
2234,1C3H,CA,Ca²⁺,10,4,2,6
2235,7RWX,ZN,Zn²⁺,3,1,2,2
2236,6PN8,ZN,Zn²⁺,4,1,2,4
2237,6SAG,CA,Ca²⁺,2,3,1,2


In [19]:
#out['info_het']
out['info_get']
for i in out['info_get']:
    if i['name'] == 'ZN' and i
        print(i)

SyntaxError: expected ':' (3413180733.py, line 4)

In [17]:
out['info_het']

[{'idx': 7762, 'atom_id': 'CU', 'atom_type': 'CU', 'name': 'CU'},
 {'idx': 7763, 'atom_id': 'CU', 'atom_type': 'CU', 'name': 'CU'},
 {'idx': 7764, 'atom_id': 'FE', 'atom_type': 'FE', 'name': 'HEC'},
 {'idx': 7765, 'atom_id': 'CHA', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7766, 'atom_id': 'CHB', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7767, 'atom_id': 'CHC', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7768, 'atom_id': 'CHD', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7769, 'atom_id': 'NA', 'atom_type': 'N', 'name': 'HEC'},
 {'idx': 7770, 'atom_id': 'C1A', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7771, 'atom_id': 'C2A', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7772, 'atom_id': 'C3A', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7773, 'atom_id': 'C4A', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7774, 'atom_id': 'CMA', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7775, 'atom_id': 'CAA', 'atom_type': 'C', 'name': 'HEC'},
 {'idx': 7776, 'atom_id': 'CBA', 'atom_type': 'C', 'name': 'HEC'}

In [15]:
out['info_get']


KeyError: 'info_get'

In [109]:
# pdb

In [88]:
chains = []
for idx in out['pdb_idx']:
    chains.append(idx[0])

In [26]:
out.keys()

dict_keys(['xyz', 'mask', 'idx', 'seq', 'pdb_idx', 'atom_type', 'xyz_het', 'info_het'])

In [110]:
# float((df_homo_filter['Ion'][0][-2]))#.encode('utf8')#.decode('ascii')

In [111]:
# #bytes(df_homo_filter['Ion'][0], 'cp1252').decode('utf8')#.decode('utf8')
# df_homo_filter['Ion'][0].encode('ascii','ignore')


In [112]:
# for i, resi_xyz in enumerate(out['xyz']):
#     for j, bb_xyz in enumerate(resi_xyz):
#         #print(bb_xyz)
#         if np.linalg.norm(bb_xyz) <= 3.0:
#             print(bb_xyz)
#             print(out['atom_type'][i][j])
#         if any(np.isnan(bb_xyz)):
#             #print(bb_xyz)
#             continue

In [33]:
rotamer_motif[0]

'/home/lhtran/class/scaffold_squad/datasets/rotamer_motif/1WWS_1.pdb'

In [10]:
#df_homo_filter.sort_values(by='Number of bound amino acids or nucleotide residues', ascending=False)#.head(20)
rotamer_motif[0]

'/home/lhtran/class/scaffold_squad/datasets/rotamer_motif/1WWS_1.pdb'

In [50]:
# periodic_table = pd.read_csv('periodic_table.csv')
# periodic_table.keys()

In [48]:
periodic_table_1 = pd.read_csv('periodic_table_1.csv')
periodic_table_1.keys()

Index(['AtomicNumber', 'Element', 'Symbol', 'AtomicMass', 'NumberOfNeutrons',
       'NumberOfProtons', 'NumberOfElectrons', 'Period', 'Group', 'Phase',
       'Radioactive', 'Natural', 'Metal', 'Nonmetal', 'Metalloid', 'Type',
       'AtomicRadius', 'Electronegativity', 'ionizationEnergy', 'Density',
       'MeltingPoint', 'BoilingPoint', 'stableIsotopes', 'Discoverer', 'Year',
       'SpecificHeat', 'NumberOfShells', 'NumberOfValence'],
      dtype='object')

In [49]:
periodic_table_1[periodic_table_1['Symbol'] == 'N']['AtomicRadius']

6    155.0
Name: AtomicRadius, dtype: float64

In [43]:
#periodic_table['NumberofValence']
periodic_table[periodic_table['Symbol'] =='N']['AtomicRadius']

6    0.75
Name: AtomicRadius, dtype: float64

In [29]:
line='ATOM   3971  CD1 TRP B 254      -2.310   1.761  -6.638  1.00 12.88           C'
line[76:78] .strip()

'C'

In [113]:
tmp

,pdb,pdb_id,metal,ion,bound_residue,num_bound_residue,paper_bound_residue,num_paper_bound_residue,num_bound_chain,motif_binding_xyz,num_NN_atom,NN_atom_properties
0,1WWS_1.pdb,1WWS,CA,Ca²⁺,['Y'],1,TYR.,1,1,"{'xyz': array([[[-1.849, -6.217, 5.614],\n ...",6,"{'atom_type': ['O'], 'l2_d': [2.8696444], 'van..."
1,4K9N_1.pdb,4K9N,MG,Mg²⁺,"['N', 'L', 'R', 'N', 'L', 'R']",6,ARG.ASN.LEU.,3,2,"{'xyz': array([[[ 5.206e+00, -6.530e-01, -3.41...",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2,2A0Q_1.pdb,2A0Q,K,K⁺,"['D', 'D', 'I', 'G']",4,ASP.GLY.ILE.,3,2,"{'xyz': array([[[-4.524, -2.809, 1.515],\n ...",6,"{'atom_type': ['O', 'O', 'O'], 'l2_d': [2.9466..."
3,3E7Z_1.pdb,3E7Z,ZN,Zn²⁺,['H'],1,HIS.,1,1,"{'xyz': array([[[-4.842, 3.076, -2.066],\n ...",6,"{'atom_type': ['N'], 'l2_d': [2.068424], 'van_..."
4,7MCI_1.pdb,7MCI,CA,Ca²⁺,"['D', 'D', 'R', 'E']",4,ARG.ASP.GLU.,3,2,"{'xyz': array([[[-1.000e-03, 5.978e+00, -3.13...",6,"{'atom_type': ['O', 'O', 'O', 'O'], 'l2_d': [2..."
...,...,...,...,...,...,...,...,...,...,...,...,...
2234,1C3H_1.pdb,1C3H,CA,Ca²⁺,"['N', 'V', 'D', 'Q', 'V', 'D', 'Q', 'Q', 'N', ...",10,ASN.ASP.GLN.VAL.,4,2,"{'xyz': array([[[ 3.755, 0.878, 4.795],\n ...",6,"{'atom_type': ['O', 'O', 'O', 'O', 'O', 'O'], ..."
2235,7RWX_1.pdb,7RWX,ZN,Zn²⁺,"['H', 'H', 'H']",3,HIS.,1,2,"{'xyz': array([[[-6.217, -2.745, -2.579],\n ...",6,"{'atom_type': ['N', 'N'], 'l2_d': [2.1764033, ..."
2236,6PN8_1.pdb,6PN8,ZN,Zn²⁺,"['C', 'C', 'C', 'C']",4,CYS.,1,2,"{'xyz': array([[[-3.448, 2.842, -3.131],\n ...",6,"{'atom_type': ['S', 'S', 'S', 'S'], 'l2_d': [2..."
2237,6SAG_1.pdb,6SAG,CA,Ca²⁺,"['N', 'P']",2,ASN.GLU.PRO.,3,1,"{'xyz': array([[[ 3.386e+00, 5.400e-02, 5.46...",6,"{'atom_type': ['O', 'O'], 'l2_d': [2.544683, 2..."


In [2]:
df_rotamer = pd.read_csv('df_rotamer.csv')

In [12]:
import json
import yaml

In [10]:
df_rotamer['motif_binding_xyz'][0]

"{'xyz': array([[[-1.849, -6.217,  5.614],\n        [-1.722, -5.22 ,  4.553],\n        [-0.399, -4.468,  4.672],\n        [ 0.209, -4.082,  3.667],\n        [-2.889, -4.227,  4.63 ],\n        [-2.819, -3.114,  3.601],\n        [-2.095, -1.942,  3.848],\n        [-3.494, -3.226,  2.386],\n        [-2.053, -0.902,  2.903],\n        [-3.459, -2.201,  1.439],\n        [-2.744, -1.043,  1.705],\n        [-2.759, -0.016,  0.789],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan],\n        [   nan,    nan,    nan]]], dtype=float32), 'mask': array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,\n         True,  True,  True, False, False, False, Fals

In [24]:
# util.aa2num[util.]
# util.THREE_TO_ONE['A
#      ']
eval(df_rotamer['motif_binding_xyz'].values[0])

NameError: name 'array' is not defined

In [31]:
json.loads((df_rotamer['motif_binding_xyz'].values[0].split('\n')))

TypeError: the JSON object must be str, bytes or bytearray, not list

In [21]:
# from ast import literal_eval


In [22]:
# literal_eval(df_rotamer['motif_binding_xyz'][0])

ValueError: malformed node or string on line 1: <ast.Call object at 0x7fd168cd7df0>

In [ ]:
util.one_hot_aa(util.ONE_TO_THREE['C'])

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])